In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
# Load data into DataFrames
train_df = pd.read_csv(train_file_path, sep='\t', header=None, names=["label", "message"])
test_df = pd.read_csv(test_file_path, sep='\t', header=None, names=["label", "message"])

# Convert labels to 0/1
train_df["label"] = train_df["label"].map({"ham": 0, "spam": 1})
test_df["label"] = test_df["label"].map({"ham": 0, "spam": 1})

# Tokenize text
tokenizer = tf.keras.preprocessing.text.Tokenizer(oov_token="<OOV>")
tokenizer.fit_on_texts(train_df["message"])

X_train = tokenizer.texts_to_sequences(train_df["message"])
X_test = tokenizer.texts_to_sequences(test_df["message"])

# Pad sequences
max_length = 100
X_train_padded = tf.keras.preprocessing.sequence.pad_sequences(X_train, maxlen=max_length, padding='post')
X_test_padded = tf.keras.preprocessing.sequence.pad_sequences(X_test, maxlen=max_length, padding='post')

y_train = train_df["label"].values
y_test = test_df["label"].values


In [ ]:
# Build a stronger model using Bidirectional LSTM
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=10000, output_dim=64, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train_padded, y_train, epochs=5, validation_data=(X_test_padded, y_test))


In [ ]:
def predict_message(pred_text):
  sequence = tokenizer.texts_to_sequences([pred_text])
  padded = tf.keras.preprocessing.sequence.pad_sequences(sequence, maxlen=max_length, padding='post')
  prediction = model.predict(padded)[0][0]

  # FINAL THRESHOLD TWEAK
  label = "spam" if prediction > 0.02 else "ham"
  return [float(prediction), label]


In [ ]:
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    print(" Message:", msg)
    print(" Prediction:", prediction[1], f"(Score: {prediction[0]:.4f})")
    print(" Expected:", ans)
    if prediction[1] != ans:
      passed = False
      print(" WRONG ")
    else:
      print(" CORRECT ")
    print("-" * 50)

  if passed:
    print(" You passed the challenge. Great job!")
  else:
    print(" You haven't passed yet. Keep trying.")

# Run it
test_predictions()
